### **0420最新版本，注释的不用跑，后续要跑，prompt已改**

In [ ]:
# import torch
# from peft import PeftModel
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import os

# # 设置模型路径
# base_model_path = "/root/.cache/LLMS/hub/hub/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B"
# lora_model_path = "/root/LLM_news_emo_analyze/outputs/checkpoint-5534"
# merged_model_path = "/root/LLM_news_emo_analyze/DeepSeek_output/DeepSeek-R1-NewsEmo-Merged2"

# # 创建保存目录
# os.makedirs(merged_model_path, exist_ok=True)

# print("正在加载基础模型...")
# # 加载基础模型
# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_path,
#     device_map="auto",
#     torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
#     trust_remote_code=True
# )

# # 加载tokenizer并添加特殊标记
# print("正在加载tokenizer...")
# tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
# special_tokens = {"additional_special_tokens": ["<think>", "</think>"]}
# num_added_tokens = tokenizer.add_special_tokens(special_tokens)
# print(f"添加了 {num_added_tokens} 个特殊标记到tokenizer")

# # 调整模型以适应新的token
# base_model.resize_token_embeddings(len(tokenizer))

# print("正在加载LoRA模型...")
# # 加载LoRA模型
# model = PeftModel.from_pretrained(base_model, lora_model_path)

# print("正在合并模型...")
# # 合并模型
# merged_model = model.merge_and_unload()

# print(f"正在保存合并后的模型到 {merged_model_path}...")
# # 保存合并后的模型和tokenizer
# merged_model.save_pretrained(merged_model_path)
# tokenizer.save_pretrained(merged_model_path)

# print("模型合并完成！")

正在加载基础模型...


KeyboardInterrupt: 

In [2]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

/root/miniconda3/envs/deepseek_sft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 全局变量存储加载的模型和tokenizer
global_model = None
global_tokenizer = None

def load_model(model_path="/root/LLM_news_emo_analyze/DeepSeek_output/DeepSeek-R1-NewsEmo-Merged"):
    """
    加载模型，只需调用一次
    
    参数:
        model_path: 合并后模型的路径
    """
    global global_model, global_tokenizer
    
    print("正在加载模型...")
    # 加载模型和tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    
    # 确保tokenizer有正确的pad_token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # 保存到全局变量
    global_model = model
    global_tokenizer = tokenizer
    
    print("模型加载完成！")

In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
def predict(prompt, temperature=0.1, max_length=4096, return_full=False):
    """
    预测函数，自动拼接格式并返回结果
    
    参数:
        prompt: 输入的问题或指令
        temperature: 温度参数，控制生成的随机性，默认0.7
        max_length: 最大生成长度，默认2048
        return_full: 是否返回完整结果包含思考过程，默认False（只返回最终答案）
        
    返回:
        如果return_full=False，返回最终答案字符串
        如果return_full=True，返回包含思考过程和最终答案的字典
    """
    global global_model, global_tokenizer
    
    # 检查模型是否已加载
    if global_model is None or global_tokenizer is None:
        load_model()
    
    # 格式化输入，与训练数据格式一致
    formatted_prompt = f"指令: {prompt}\n\n输出: "
    
    # 编码输入
    inputs = global_tokenizer(formatted_prompt, return_tensors="pt").to(global_model.device)
    
    # 生成回答
    with torch.no_grad():
        outputs = global_model.generate(
            inputs.input_ids,
            max_length=max_length,
            temperature=temperature,
            top_p=0.9,
            do_sample=False,
            pad_token_id=global_tokenizer.pad_token_id,
            repetition_penalty=1.1,
            # no_repeat_ngram_size=3,
            eos_token_id=global_tokenizer.eos_token_id
        )
    
    # 解码输出
    full_response = global_tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    response = full_response.split("输出: ")[-1]
    
    return response

### **我自己的一个案例测试，你可以看看这个prompt格式**

In [23]:
prompt = """**角色定义**
擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分，请思考后回答。

**核心任务**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词
3. 五档制评分（-1/-0.5/0/0.5/1）

**评分逻辑**
1. 语义匹配：忽略偏离词汇，保留有效情绪词
2. 评分验证：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1（积极）：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
输出:<think>语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破。“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。</think> {"情绪得分":"1"}

- 示例2（比较积极）：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
输出:<think>语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号。此外，新闻文本未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。</think> {"情绪得分":"0.5"}

- 示例3（中性）：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
输出:<think>语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响。此外，新闻文本中“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0。</think> {"情绪得分":"0"}

- 示例4（比较消极）：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
输出:<think>语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪。新闻文本中“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位-0.5。</think> {"情绪得分":"-0.5"}

- 示例5（消极）：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
输出:<think>语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词验证：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，符合-1的情绪评分。</think> {"情绪得分":"-1"}


**输出规范**
- 得分五档制强制选择
- 输出格式：{"情绪得分":"..."}

现在请你开始进行分析，严格按照输出规范输出结果，情绪词典分值仅作语义方向参考：
新闻文本：而福喜集团总部所在地的美国，更被称作是全球食品安全监管体系最完善的国家之一，其着力于事前监管，把对食品安全的监管起点推进到田间和生产商，并且，由中央财政统一拨款，独立的检验运作核算功能，使不法企业难以蠢蠢欲动
情绪词典：perfect:1.0, food:0.0"""

In [21]:
load_model()

正在加载模型...
模型加载完成！


In [25]:
full_result = predict(prompt, return_full=True)
print(full_result)

<think>好的，我现在需要分析给定的新闻文本，并根据提供的情绪词典和评分逻辑给出五档制的情绪得分。首先，我得仔细阅读新闻内容，理解其整体情绪倾向。

新闻内容提到福喜集团总部在美国，该国是全球食品安全监管最完善的地方。他们通过事前监管、独立检验和中央财政拨款，确保食品安全，防止不法企业存在。这里的关键点在于强调了安全监管的完善和独立运作，以及中央财政的统一拨款，这些都显示出对安全措施的重视和严格执行。

接下来，检查情绪词典里的关键词。情绪词典里有两个词：perfect（1.0）和food（0.0）。新闻中没有直接出现这两个词，但可能隐含其中。例如，“perfect”暗示了高效率和严格的监管，这可能对应情绪词中的positive方向。而“food”虽然在词典中是中性，但在新闻中并没有明显的负面词汇，所以可能不太适用。

然后看评分逻辑。第一步是语义匹配，忽略偏离词汇。新闻中提到的是“食品安全监管体系最完善”，这里的“完善”可能对应“perfect”的高分，即0.6或者更高。第二步是评分验证，整体语义优先，情绪词辅助修正。如果新闻整体情绪偏向积极，情绪词的匹配可能会加强这一判断。

结合以上分析，新闻主要是在描述一个成功的食品安全监管体系，强调了严格和高效，这属于积极的情绪。因此，情绪得分应该在中性基础上稍微偏积极，比如0.5或者0.6。

不过，再仔细想想，新闻中提到的是“不法企业难以蠢蠢欲动”，也就是避免不法行为，这可能暗示了一种积极的监管环境，但整体情绪是否是中性呢？或者是否有其他因素？

另外，新闻中提到的是“中央财政统一拨款”，这可能意味着监管机构的统一协调，增加了透明性和公正性，这也是积极的信号。综合来看，整体情绪应该是积极的，但需要确认是否符合中性评分的范围。

根据评分逻辑，虽然没有直接匹配情绪词，但“完善”和“统一拨款”可能暗示积极的监管措施，所以情绪词典中的“perfect”可能增强积极因素，但整体情绪还是中性的，因为没有明显的负面词汇。

综上所述，情绪得分应为0.5。
</think> {"情绪得分":"0.5"}<｜end▁of▁sentence｜>


In [17]:
import re
score_pattern = r'{"情绪得分"\s*:\s*"([^"]+)"}'

match = re.search(score_pattern, full_result)
if match:
    score_str = match.group(1)
    print(float(score_str))

### **lhl单程尝试**

In [ ]:
import pandas as pd
import re
import time
from tqdm import tqdm

def process_dataframe_simple(df, prompt_template, max_retries=3, print_interval=100):
    """
    简单循环处理DataFrame，一行一行调用predict函数，并定期打印结果
    
    参数:
        df: 输入的DataFrame
        prompt_template: 包含{cut_news}和{sentiment_dict_v2}占位符的模板
        max_retries: 最大重试次数，默认为3
        print_interval: 打印结果的间隔，默认每100条打印一次
        
    返回:
        处理后的DataFrame，包含预测结果和提取的分数
    """
    # 添加结果列
    if 'lhl_ds_output' not in df.columns:
        df['lhl_ds_output'] = None
    if 'findal_score' not in df.columns:
        df['findal_score'] = None
    if 'retry_count' not in df.columns:
        df['retry_count'] = 0
    
    # 定义正则表达式模式，用于提取情绪得分
    score_pattern = r'{"情绪得分"\s*:\s*"([^"]+)"}'
    
    # 创建进度条
    print(f"开始处理 {len(df)} 行数据...")
    for idx in tqdm(range(len(df))):
        row = df.iloc[idx]
        retry_count = 0
        
        while retry_count < max_retries:
            try:
                
                current_prompt = prompt_template.replace("{cut_news}", str(row['cut_news'])).replace("{sentiment_dict_v2}", str(row['sentiment_dict_v2']))
                # print(current_prompt)
                
                # 调用predict函数进行预测
                temperature = 0.1
                response = predict(current_prompt, temperature=temperature)
                # print(response)
                
                # 保存完整输出
                df.at[idx, 'lhl_ds_output'] = response
                
                # 提取情绪得分
                match = re.search(score_pattern, response)
                if match:
                    score_str = match.group(1)
                    try:
                        # 尝试转换为浮点数
                        score = float(score_str)
                        # 检查是否为有效得分
                        if score in [-1.0, -0.5, 0.0, 0, 0.5, 1.0]:
                            df.at[idx, 'findal_score'] = score
                            df.at[idx, 'retry_count'] = retry_count
                            break  # 成功提取，退出重试循环
                        else:
                            # 记录重试次数
                            retry_count += 1
                    except ValueError:
                        # 无法转换为浮点数，记录重试次数
                        retry_count += 1
                else:
                    # 未找到匹配，记录重试次数
                    retry_count += 1
                
            except Exception as e:
                print(f"处理第 {idx} 行时出错: {str(e)}")
                df.at[idx, 'lhl_ds_output'] = f"ERROR: {str(e)}"
                retry_count += 1
                time.sleep(1)  # 出错后稍作等待
                
            # 清理GPU缓存
            torch.cuda.empty_cache()
            
            # 如果已经重试了最大次数但仍未成功提取分数
            if retry_count >= max_retries:
                df.at[idx, 'retry_count'] = retry_count
                if pd.isna(df.at[idx, 'findal_score']):
                    df.at[idx, 'findal_score'] = 'FAILED_AFTER_RETRIES'
                break
        
        # 每隔print_interval条数据打印一次结果
        if (idx + 1) % print_interval == 0 or idx == len(df) - 1:
            print("\n" + "="*80)
            print(f"处理进度: {idx + 1}/{len(df)} ({(idx + 1)/len(df)*100:.2f}%)")
            
            # 计算当前的成功率
            current_success_count = len(df.iloc[:idx+1][df.iloc[:idx+1]['findal_score'].apply(lambda x: isinstance(x, float))])
            print(f"当前成功率: {current_success_count}/{idx + 1} ({current_success_count/(idx + 1)*100:.2f}%)")
            
            # 打印最近处理的这条记录
            print("\n最新处理记录:")
            print(f"索引: {idx}")
            print(f"新闻文本: {str(row['cut_news'])[:100]}..." if len(str(row['cut_news'])) > 100 else str(row['cut_news']))
            print(f"情绪词典: {str(row['sentiment_dict_v2'])[:100]}..." if len(str(row['sentiment_dict_v2'])) > 100 else str(row['sentiment_dict_v2']))
            print(f"模型输出: {df.at[idx, 'lhl_ds_output']}")
            print(f"提取的分数: {df.at[idx, 'findal_score']}")
            print(f"重试次数: {df.at[idx, 'retry_count']}")
            print("="*80 + "\n")
    
    # 统计最终成功率
    success_count = len(df[df['findal_score'].apply(lambda x: isinstance(x, float))])
    print(f"\n全部处理完成！成功率: {success_count}/{len(df)} ({success_count/len(df)*100:.2f}%)")
    print(f"平均重试次数: {df['retry_count'].mean():.2f}")
    
    return df

# 使用示例
if __name__ == "__main__":
    # 确保已经导入必要的库和定义了predict函数
    
    # 读取数据
    df = pd.read_csv("/root/LLM_news_emo_analyze/wxy_use/deepseek_test_500_wxy_0420.csv", encoding='utf_8_sig', lineterminator='\n')
    
    # 设置prompt模板
    prompt_template = """**角色定义**
擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分，请思考后回答。

**核心任务**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词
3. 五档制评分（-1/-0.5/0/0.5/1）

**评分逻辑**
1. 语义匹配：忽略偏离词汇，保留有效情绪词
2. 评分验证：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1（积极）：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
输出:<think>语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破。“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。</think> {"情绪得分":"1"}

- 示例2（比较积极）：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
输出:<think>语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号。此外，新闻文本未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。</think> {"情绪得分":"0.5"}

- 示例3（中性）：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
输出:<think>语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响。此外，新闻文本中“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0。</think> {"情绪得分":"0"}

- 示例4（比较消极）：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
输出:<think>语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪。新闻文本中“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位-0.5。</think> {"情绪得分":"-0.5"}

- 示例5（消极）：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
输出:<think>语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词验证：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，符合-1的情绪评分。</think> {"情绪得分":"-1"}


**输出规范**
- 得分五档制强制选择
- 输出格式：{"情绪得分":"..."}

现在请你开始进行分析，严格按照输出规范输出结果，情绪词典分值仅作语义方向参考：
新闻文本：{cut_news}
情绪词典：{sentiment_dict_v2}"""
    
    # 处理数据，每100条打印一次
    processed_df = process_dataframe_simple(df, prompt_template, max_retries=3, print_interval=100)
    
    # 保存结果
    processed_df.to_csv("processed_data_simple.csv", index=False)

开始处理 500 行数据...


  0%|          | 0/500 [00:00<?, ?it/s]/root/miniconda3/envs/deepseek_sft/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/deepseek_sft/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
 20%|██        | 100/500 [1:12:35<1:51:46, 16.77s/it] 


处理进度: 100/500 (20.00%)
当前成功率: 80/100 (80.00%)

最新处理记录:
索引: 99
@mufeng622：一边实名购票，一边“黄牛”遍地，搞笑至极
fare:0.4, problematic:-0.5
模型输出: <think>好的，我现在需要处理这个用户的查询，分析给定的新闻文本的情绪，并给出五档制评分。首先，我得仔细阅读用户提供的所有信息，确保自己完全理解任务的要求和评分逻辑。

首先看角色定义部分，用户希望我作为中英双语的市场情绪分析专家，对中文市场的新闻文本进行情绪评分。核心任务包括三个步骤：分析整体内容情绪、匹配情绪词典关键词，以及按照评分逻辑给出五档分数。评分逻辑分为语义匹配和评分验证两个部分。

接下来，看看示例，比如示例1是关于中信证券成为业内第一的新闻，情绪词典中有profitability和profit的高分词。输出是1分，因为这两词都支持积极的结果。而示例2则是关于A股被贴价，虽然没有直接的匹配词，但整体情绪是乐观的，所以评分是0.5分。

现在来看具体的新闻文本：“@mufeng622：一边实名购票，一边‘黄牛’遍地，搞笑至极。” 这里的关键点在于“实名购票”和“‘黄牛’遍地”。实名购票通常意味着身份验证，可能带来信任感，而“黄牛”通常指不道德的人，可能暗示不道德行为。这里“黄牛”可能被用来比喻某些人或行为，导致负面情绪。

情绪词典里有“fear”对应-0.5，但这里并没有直接出现这个词。不过“黄牛”可能隐含负面情绪，因为“黄牛”通常指的是不道德的人，可能在社交媒体上被用来形容某些人的行为，导致负面情绪。因此，结合整体语义，这可能是一个中性或者稍微负面的新闻。

接下来检查是否有匹配情绪词典中的关键词。新闻中没有提到任何情绪词典中的词，比如“mispricing”或“advantage”，所以情绪词典中的词可能无法提供有效的匹配。但是，整体语义是带有负面情绪的，因为“黄牛”可能暗示负面行为，但具体是否达到中性或负面还需要看上下文。

根据评分逻辑，第一步是语义匹配。这里没有直接匹配的情绪词典中的词，但“黄牛”可能暗示负面情绪，所以情绪词可能有一定的负面倾向。第二步是评分验证，整体语义是负面的，但缺乏明确的负面词典词，所以可能只能处于中性或稍微负面的位置。

综合来看，这条新闻传达的是负面情绪，特别是

 40%|████      | 200/500 [2:26:45<2:18:46, 27.75s/it]  


处理进度: 200/500 (40.00%)
当前成功率: 161/200 (80.50%)

最新处理记录:
索引: 199
亚太经济区域是全世界经济发展最为活跃的地区，而澳门虽然是一个微型经济体，但由于经济实力的快速增长，成为该区域内一个引人注目的亮点
economy:0.0, region:0.0
模型输出: <think>好的，我现在需要分析给定的新闻文本，并根据提供的情绪词典和评分逻辑给出五档制评分。首先，我得仔细阅读新闻内容，理解其整体情绪。

新闻文本是关于亚太经济区域和澳门作为微型经济体的情况。提到澳门因为经济实力快速增长而成为亮点，这可能暗示其在经济领域具有吸引力。接下来，检查情绪词典里的关键词，这里有两个词：economy和region，对应的分数都是0.0。

然后按照评分逻辑，先看整体语义。新闻中提到经济实力快速增长，这通常是积极的信号，比如经济繁荣、增长潜力大等。虽然澳门作为一个微型经济体，但其快速发展的速度可能意味着其经济竞争力增强，属于积极因素。

接下来看情绪词典中的关键词是否匹配。这里没有直接出现“economy”或“region”的具体词汇，只是用了一个“经济实力快速增长”。不过，虽然没有具体的词汇，但整体语义上已经显示出经济活力，属于积极的情绪。

所以，综合考虑，新闻文本传达的是一个积极的发展趋势，经济实力快速增长，这可能带来经济繁荣和增长潜力，符合中性评分的一个方向。但因为没有明显的负面词汇，整体情绪还是偏向积极的。

因此，评分应该在中性范围内，即0.0。
</think> {"情绪得分":"0"}<｜end▁of▁sentence｜>
提取的分数: 0.0
重试次数: 0



 42%|████▏     | 209/500 [2:34:49<3:14:51, 40.18s/it]

In [ ]:
# import pandas as pd
# import re
# import time
# import torch
# from tqdm import tqdm
# from concurrent.futures import ThreadPoolExecutor

# def batch_predict(prompts, temperature=0.3, max_length=2048):
#     """
#     批量预测函数，同时处理多个提示
    
#     参数:
#         prompts: 提示列表
#         temperature: 温度参数
#         max_length: 最大生成长度
        
#     返回:
#         预测结果列表
#     """
#     global global_model, global_tokenizer
    
#     # 检查模型是否已加载
#     if global_model is None or global_tokenizer is None:
#         load_model()
    
#     # 格式化输入
#     formatted_prompts = [f"指令: {prompt}\n\n输出: " for prompt in prompts]
    
#     # 编码输入 - 批量处理
#     inputs = global_tokenizer(
#         formatted_prompts, 
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=2048
#     ).to(global_model.device)
    
#     # 生成回答
#     with torch.inference_mode():
#         outputs = global_model.generate(
#             inputs.input_ids,
#             max_length=max_length,
#             temperature=temperature,
#             top_p=0.9,
#             # do_sample=True,
#             pad_token_id=global_tokenizer.pad_token_id,
#             repetition_penalty=1.03,
#             eos_token_id=global_tokenizer.eos_token_id
#         )
    
#     # 解码输出
#     results = [
#         global_tokenizer.decode(output, skip_special_tokens=False, clean_up_tokenization_spaces=False)
#         for output in outputs
#     ]
    
#     return results

# def process_dataframe(df, prompt_template, batch_size=16, max_retries=3):
#     """
#     处理DataFrame，批量生成预测并提取分数，失败时重试
    
#     参数:
#         df: 输入的DataFrame
#         prompt_template: 包含{cut_news}和{sentiment_dict_v2}占位符的模板
#         batch_size: 批处理大小，默认8
#         max_retries: 最大重试次数，默认3
        
#     返回:
#         处理后的DataFrame，包含预测结果和提取的分数
#     """
#     # 添加结果列
#     if 'lhl_ds_output' not in df.columns:
#         df['lhl_ds_output'] = None
#     if 'findal_score' not in df.columns:
#         df['findal_score'] = None
#     if 'retry_count' not in df.columns:
#         df['retry_count'] = 0
    
#     # 定义正则表达式模式，用于提取情绪得分
#     score_pattern = r'{"情绪得分"\s*:\s*"([^"]+)"}'
    
#     # 计算需要处理的批次数
#     total_rows = len(df)
#     num_batches = (total_rows + batch_size - 1) // batch_size
    
#     print(f"开始处理 {total_rows} 行数据，分 {num_batches} 个批次处理...")
    
#     # 创建重试队列
#     retry_indices = []
    
#     # 批量处理数据
#     for batch_idx in tqdm(range(num_batches)):
#         start_idx = batch_idx * batch_size
#         end_idx = min(start_idx + batch_size, total_rows)
        
#         # 准备当前批次的提示
#         batch_prompts = []
#         batch_indices = []
        
#         for i in range(start_idx, end_idx):
#             row = df.iloc[i]
#             # 仅处理尚未成功提取分数的行
#             if pd.isna(df.at[i, 'findal_score']) and df.at[i, 'retry_count'] < max_retries:
#                 # 替换模板中的占位符
#                 current_prompt = prompt_template.replace("{cut_news}", str(row['cut_news'])).replace("{sentiment_dict_v2}", str(row['sentiment_dict_v2']))
#                 batch_prompts.append(current_prompt)
#                 batch_indices.append(i)
        
#         # 如果没有需要处理的提示，跳过当前批次
#         if not batch_prompts:
#             continue
        
#         try:
#             # 批量生成预测
#             predictions = batch_predict(batch_prompts, temperature=0.3)
#             # print(predictions)
            
#             # 处理预测结果
#             for idx, prediction in zip(batch_indices, predictions):
#                 # 保存完整输出
#                 df.at[idx, 'lhl_ds_output'] = prediction
                
#                 # 提取情绪得分
#                 match = re.search(score_pattern, prediction)
#                 if match:
#                     score_str = match.group(1)
#                     try:
#                         # 尝试转换为浮点数
#                         score = float(score_str)
#                         df.at[idx, 'findal_score'] = score
#                         # 成功提取，不需要重试
#                     except ValueError:
#                         # 无法转换为浮点数，需要重试
#                         df.at[idx, 'retry_count'] += 1
#                         if df.at[idx, 'retry_count'] < max_retries:
#                             retry_indices.append(idx)
#                 else:
#                     # 未找到匹配，需要重试
#                     df.at[idx, 'retry_count'] += 1
#                     if df.at[idx, 'retry_count'] < max_retries:
#                         retry_indices.append(idx)
            
#             # 清理CUDA缓存
#             torch.cuda.empty_cache()
                
#         except Exception as e:
#             print(f"处理批次 {batch_idx} 时出错: {str(e)}")
#             # 将整个批次添加到重试队列
#             for idx in batch_indices:
#                 df.at[idx, 'lhl_ds_output'] = f"ERROR: {str(e)}"
#                 df.at[idx, 'retry_count'] += 1
#                 if df.at[idx, 'retry_count'] < max_retries:
#                     retry_indices.append(idx)
            
#             # 清理CUDA缓存
#             torch.cuda.empty_cache()
#             time.sleep(2)  # 出错后稍作等待
    
#     # 处理重试队列
#     if retry_indices:
#         print(f"处理 {len(retry_indices)} 条需要重试的数据...")
        
#         # 对需要重试的行单独处理
#         for idx in tqdm(retry_indices):
#             row = df.iloc[idx]
#             remaining_retries = max_retries - df.at[idx, 'retry_count']
            
#             for retry in range(remaining_retries):
#                 try:
#                     # 替换模板中的占位符，增加重试提示
#                     current_prompt = prompt_template.replace("{cut_news}", str(row['cut_news'])).replace("{sentiment_dict_v2}", str(row['sentiment_dict_v2']))
                    
#                     # 使用较低温度增加确定性
#                     prediction = predict(current_prompt, temperature=0.3)
#                     prediction = prediction.split("输出: ")[-1]
                    
#                     # 保存完整输出
#                     df.at[idx, 'lhl_ds_output'] = prediction
                    
#                     # 提取情绪得分
#                     match = re.search(score_pattern, prediction)
#                     if match:
#                         score_str = match.group(1)
#                         try:
#                             score = float(score_str)
#                             df.at[idx, 'findal_score'] = score
#                             # 成功提取，不需要继续重试
#                             break
#                         except ValueError:
#                             # 记录重试次数
#                             df.at[idx, 'retry_count'] += 1
#                             continue
#                     else:
#                         # 记录重试次数
#                         df.at[idx, 'retry_count'] += 1
#                         continue
                    
#                 except Exception as e:
#                     print(f"重试第 {idx} 行时出错: {str(e)}")
#                     df.at[idx, 'retry_count'] += 1
#                     time.sleep(1)  # 出错后稍作等待
                
#                 # 每次重试后清理CUDA缓存
#                 torch.cuda.empty_cache()
    
#     # 标记最终仍无法提取分数的行
#     for i in range(len(df)):
#         if pd.isna(df.at[i, 'findal_score']) and df.at[i, 'retry_count'] >= max_retries:
#             df.at[i, 'findal_score'] = 'FAILED_AFTER_RETRIES'
    
#     print("处理完成！")
#     return df

# # 使用示例
# if __name__ == "__main__":
#     # 加载模型（仅需一次）
#     load_model()
    
#     # 读取数据
#     df = pd.read_csv("/root/LLM_news_emo_analyze/wxy_use/deepseek_test_500_wxy_0420.csv", encoding='utf_8_sig', lineterminator='\n')
    
#     # 设置prompt模板
#     prompt_template = """**角色定义**
# 擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分，请思考后回答。

# **核心任务**
# 1. 分析新闻整体内容情绪
# 2. 匹配情绪词典关键词
# 3. 五档制评分（-1/-0.5/0/0.5/1）

# **评分逻辑**
# 1. 语义匹配：忽略偏离词汇，保留有效情绪词
# 2. 评分验证：整体语义优先，情绪词辅助修正

# **示例说明**
# - 示例1（积极）：
# 新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
# 情绪词典：profitability:0.6,profit:0.8
# 输出:<think>语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破。“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。</think> {"情绪得分":"1"}

# - 示例2（比较积极）：
# 新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
# 情绪词典：mispricing:-0.4,advantage:0.7
# 输出:<think>语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号。此外，新闻文本未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。</think> {"情绪得分":"0.5"}

# - 示例3（中性）：
# 新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
# 情绪词典：free:0.2,immateriality:-0.2
# 输出:<think>语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响。此外，新闻文本中“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0。</think> {"情绪得分":"0"}

# - 示例4（比较消极）：
# 新闻文本：我们投入的前期费用谁来承担
# 情绪词典：invest:0.3
# 输出:<think>语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪。新闻文本中“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位-0.5。</think> {"情绪得分":"-0.5"}

# - 示例5（消极）：
# 新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
# 情绪词典：crash:-0.9,meltdown:-0.8
# 输出:<think>语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词验证：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，符合-1的情绪评分。</think> {"情绪得分":"-1"}


# **输出规范**
# - 五档制强制选择
# - 输出格式：{"情绪得分":"..."}

# 现在请你开始进行分析，严格按照输出规范输出结果，情绪词典分值仅作语义方向参考：
# 新闻文本：{cut_news}
# 情绪词典：{sentiment_dict_v2}"""
    
#     # 使用批处理加速推理并支持重试逻辑
#     processed_df = process_dataframe(df, prompt_template, batch_size=48, max_retries=3)
    
#     # 保存结果
#     processed_df.to_csv("processed_data_with_retries.csv", index=False,encoding = 'utf_8_sig')
    
#     # 分析成功率
#     success_count = len(processed_df[processed_df['findal_score'].apply(lambda x: isinstance(x, float))])
#     print(f"成功率: {success_count}/{len(processed_df)} ({success_count/len(processed_df)*100:.2f}%)")

正在加载模型...
模型加载完成！
开始处理 500 行数据，分 11 个批次处理...


100%|██████████| 11/11 [13:14<00:00, 72.25s/it]


处理完成！
成功率: 500/500 (100.00%)


In [4]:
import pandas as pd
import re
import time
import torch
from tqdm import tqdm

def batch_process_dataframe(df, prompt_template, batch_size=4, max_retries=3):
    """
    使用批处理方式处理DataFrame，提高GPU利用率
    
    参数:
        df: 输入的DataFrame
        prompt_template: 包含{cut_news}和{sentiment_dict_v2}占位符的模板
        batch_size: 批处理大小，默认为4
        max_retries: 最大重试次数，默认为3
        
    返回:
        处理后的DataFrame，包含预测结果和提取的分数
    """
    global global_model, global_tokenizer
    
    # 检查模型是否已加载
    if global_model is None or global_tokenizer is None:
        load_model()
    
    # 添加结果列
    if 'lhl_ds_output' not in df.columns:
        df['lhl_ds_output'] = None
    if 'findal_score' not in df.columns:
        df['findal_score'] = None
    if 'retry_count' not in df.columns:
        df['retry_count'] = 0
    
    # 定义正则表达式模式，用于提取情绪得分
    score_pattern = r'{"情绪得分"\s*:\s*"([^"]+)"}'
    
    # 初始化索引列表和对应的提示词列表
    all_indices = []
    all_prompts = []
    
    # 准备所有需要处理的数据
    print("准备批处理数据...")
    for idx in range(len(df)):
        if pd.isna(df.at[idx, 'findal_score']) and df.at[idx, 'retry_count'] < max_retries:
            row = df.iloc[idx]
            current_prompt = prompt_template.replace("{cut_news}", str(row['cut_news'])).replace("{sentiment_dict_v2}", str(row['sentiment_dict_v2']))
            all_indices.append(idx)
            all_prompts.append(current_prompt)
    
    # 如果没有需要处理的数据，直接返回
    if not all_indices:
        print("没有需要处理的数据")
        return df
    
    # 批量处理函数
    def batch_predict(indices, prompts):
        # 格式化所有提示词
        formatted_prompts = [f"指令: {p}\n\n输出: " for p in prompts]
        
        # 编码所有输入
        inputs = global_tokenizer(
            formatted_prompts,
            padding=True,
            return_tensors="pt",
            truncation=True,
            max_length=2048  # 输入文本最大长度
        ).to(global_model.device)
        
        # 批量生成
        with torch.no_grad():
            outputs = global_model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_length=4096,
                temperature=0.1,
                top_p=0.9,
                pad_token_id=global_tokenizer.pad_token_id,
                repetition_penalty=1.03,
                eos_token_id=global_tokenizer.eos_token_id
            )
        
        # 解码所有输出
        full_responses = [global_tokenizer.decode(output, skip_special_tokens=False) for output in outputs]
        
        # 提取实际响应部分
        responses = [response.split("输出: ")[-1] for response in full_responses]
        
        return responses
    
    # 计算需要处理的批次数
    total_batches = (len(all_indices) + batch_size - 1) // batch_size
    
    # 首次批量处理所有数据
    print(f"开始处理 {len(all_indices)} 条数据，共 {total_batches} 个批次...")
    processed_count = 0
    
    for i in tqdm(range(0, len(all_indices), batch_size)):
        batch_indices = all_indices[i:i+batch_size]
        batch_prompts = all_prompts[i:i+batch_size]
        
        try:
            # 批量预测
            batch_responses = batch_predict(batch_indices, batch_prompts)
            
            # 处理预测结果
            for idx, response in zip(batch_indices, batch_responses):
                df.at[idx, 'lhl_ds_output'] = response
                
                # 提取情绪得分
                match = re.search(score_pattern, response)
                if match:
                    score_str = match.group(1)
                    try:
                        # 尝试转换为浮点数
                        score = float(score_str)
                        # 验证分数是否在有效范围内
                        if score in [-1.0, -0.5, 0.0, 0.5, 1.0]:
                            df.at[idx, 'findal_score'] = score
                            processed_count += 1
                        else:
                            # 无效分数，记录重试次数
                            df.at[idx, 'retry_count'] += 1
                    except ValueError:
                        # 无法转换为浮点数，记录重试次数
                        df.at[idx, 'retry_count'] += 1
                else:
                    # 未找到匹配，记录重试次数
                    df.at[idx, 'retry_count'] += 1
                    
        except Exception as e:
            print(f"处理批次出错: {str(e)}")
            # 标记批次中的所有样本需要重试
            for idx in batch_indices:
                df.at[idx, 'retry_count'] += 1
                df.at[idx, 'lhl_ds_output'] = f"ERROR: {str(e)}"
        
        # 每批次后清理GPU缓存
        torch.cuda.empty_cache()
    
    # 处理需要重试的数据
    retry_round = 1
    while retry_round < max_retries:
        # 收集需要重试的样本
        retry_indices = []
        retry_prompts = []
        
        for idx in range(len(df)):
            if pd.isna(df.at[idx, 'findal_score']) and df.at[idx, 'retry_count'] == retry_round:
                row = df.iloc[idx]
                # 为重试添加额外提示
                retry_prompt = prompt_template.replace("{cut_news}", str(row['cut_news'])).replace("{sentiment_dict_v2}", str(row['sentiment_dict_v2']))
                
                retry_indices.append(idx)
                retry_prompts.append(retry_prompt)
        
        if not retry_indices:
            break
            
        # 计算重试批次数
        retry_batches = (len(retry_indices) + batch_size - 1) // batch_size
        print(f"第{retry_round+1}轮重试: 处理 {len(retry_indices)} 条数据，共 {retry_batches} 个批次...")
        
        for i in tqdm(range(0, len(retry_indices), batch_size)):
            batch_indices = retry_indices[i:i+batch_size]
            batch_prompts = retry_prompts[i:i+batch_size]
            
            try:
                # 重试时使用更低的温度
                temperature = max(0.1, 0.3 - 0.1 * retry_round)
                
                # 单独处理每个样本，降低批处理中的干扰
                for idx, prompt in zip(batch_indices, batch_prompts):
                    # 使用原始predict函数进行重试
                    response = predict(prompt, temperature=temperature)
                    df.at[idx, 'lhl_ds_output'] = response
                    
                    # 提取情绪得分
                    match = re.search(score_pattern, response)
                    if match:
                        score_str = match.group(1)
                        try:
                            score = float(score_str)
                            if score in [-1.0, -0.5, 0.0, 0.5, 1.0]:
                                df.at[idx, 'findal_score'] = score
                                processed_count += 1
                            else:
                                df.at[idx, 'retry_count'] += 1
                        except ValueError:
                            df.at[idx, 'retry_count'] += 1
                    else:
                        df.at[idx, 'retry_count'] += 1
                    
                    # 每次处理后清理GPU缓存
                    torch.cuda.empty_cache()
                
            except Exception as e:
                print(f"重试批次出错: {str(e)}")
                for idx in batch_indices:
                    df.at[idx, 'retry_count'] += 1
                    df.at[idx, 'lhl_ds_output'] = f"ERROR in retry {retry_round+1}: {str(e)}"
                
                # 出错后清理GPU缓存
                torch.cuda.empty_cache()
                time.sleep(1)  # 出错后稍作等待
        
        retry_round += 1
    
    # 标记最终仍无法提取分数的行
    for idx in range(len(df)):
        if pd.isna(df.at[idx, 'findal_score']) and df.at[idx, 'retry_count'] >= max_retries:
            df.at[idx, 'findal_score'] = 'FAILED_AFTER_RETRIES'
    
    # 统计成功率
    success_count = len(df[df['findal_score'].apply(lambda x: isinstance(x, float))])
    print(f"处理完成！成功率: {success_count}/{len(df)} ({success_count/len(df)*100:.2f}%)")
    print(f"平均重试次数: {df['retry_count'].mean():.2f}")
    
    return df

# 使用示例
if __name__ == "__main__":
    # 确保已经导入必要的库和定义了predict函数
    # import torch
    # from transformers import AutoModelForCausalLM, AutoTokenizer
    
    # 加载模型（仅需一次）
    # load_model()
    
    # 读取数据
    df = pd.read_csv("/root/LLM_news_emo_analyze/wxy_use/deepseek_test_500_wxy_0420.csv", encoding='utf_8_sig', lineterminator='\n')
    
    # 设置prompt模板
    prompt_template = """**角色定义**
擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分，请思考后回答。

**核心任务**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词
3. 五档制评分（-1/-0.5/0/0.5/1）

**评分逻辑**
1. 语义匹配：忽略偏离词汇，保留有效情绪词
2. 评分验证：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1（积极）：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
输出:<think>语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破。“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。</think> {"情绪得分":"1"}

- 示例2（比较积极）：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
输出:<think>语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号。此外，新闻文本未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。</think> {"情绪得分":"0.5"}

- 示例3（中性）：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
输出:<think>语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响。此外，新闻文本中“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0。</think> {"情绪得分":"0"}

- 示例4（比较消极）：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
输出:<think>语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪。新闻文本中“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位-0.5。</think> {"情绪得分":"-0.5"}

- 示例5（消极）：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
输出:<think>语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词验证：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，符合-1的情绪评分。</think> {"情绪得分":"-1"}


**输出规范**
- 五档制强制选择
- 输出格式：{"情绪得分":"..."}

现在请你开始进行分析，严格按照输出规范输出结果，情绪词典分值仅作语义方向参考：
新闻文本：{cut_news}
情绪词典：{sentiment_dict_v2}"""
    
    # 使用批处理加速推理
    processed_df = batch_process_dataframe(df, prompt_template, batch_size=32, max_retries=3)
    
    # 保存结果
    processed_df.to_csv("processed_data_batched.csv", index=False)

准备批处理数据...
开始处理 500 条数据，共 16 个批次...


100%|██████████| 16/16 [57:22<00:00, 215.18s/it]


第2轮重试: 处理 193 条数据，共 7 个批次...


  0%|          | 0/7 [00:00<?, ?it/s]/root/miniconda3/envs/deepseek_sft/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.19999999999999998` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/deepseek_sft/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
  0%|          | 0/7 [19:14<?, ?it/s]


KeyboardInterrupt: 

In [11]:
processed_df.to_csv("processed_data_batched.csv", index=False,encoding='utf_8_sig')

In [9]:
import pandas as pd


# 记录删除前的行数
original_rows = len(processed_df)

# 删除 findal_score 列的值是 FAILED_AFTER_RETRIES 的行
processed_df2 = processed_df[processed_df['findal_score'] != 'FAILED_AFTER_RETRIES']

# 记录删除后的行数
remaining_rows = len(processed_df2)

# 计算删除的行数
deleted_rows = original_rows - remaining_rows

print(f"删除了 {deleted_rows} 行")

删除了 2 行


In [14]:
value_counts = processed_df2['人工标注'].value_counts()
print("df['findal_score'] 的数值分布：",value_counts)

df['findal_score'] 的数值分布： 人工标注
 0.0    190
 0.5    158
-0.5     80
-1.0     46
 1.0     23
Name: count, dtype: int64


In [10]:
import pandas as pd


# 统一列数据类型为浮点数
def unify_column_types(df, columns):
    for col in columns:
        df[col] = df[col].astype(float)
    return df


# 计算两列的一致率
def calculate_consistency_rate(col1, col2):
    same_count = (col1 == col2).sum()
    total_count = len(col1)
    return same_count / total_count if total_count > 0 else 0

columns_to_convert = ['deepseek', 'findal_score', '人工标注']
# 统一数据类型
processed_df2 = unify_column_types(processed_df2, columns_to_convert)

# 计算findal_score与deepseek的一致率
consistency_rate_1 = calculate_consistency_rate(processed_df2['findal_score'], processed_df2['deepseek'])
print(f"findal_score与deepseek的一致率: {consistency_rate_1}")

# 计算findal_score与人工标注的一致率
consistency_rate_2 = calculate_consistency_rate(processed_df2['findal_score'], processed_df2['人工标注'])
print(f"findal_score与人工标注的一致率: {consistency_rate_2}")

findal_score与deepseek的一致率: 0.40562248995983935
findal_score与人工标注的一致率: 0.40562248995983935


/tmp/ipykernel_7352/558342751.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)
/tmp/ipykernel_7352/558342751.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)
/tmp/ipykernel_7352/558342751.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [71]:
test.to_csv('ouutput2.csv',encoding='utf_8_sig')

In [ ]:
# import pandas as pd
# import re
# import time
# from tqdm import tqdm

# def process_dataframe(df, prompt_template, batch_size=1):
#     """
#     处理DataFrame，为每行生成预测并提取分数
    
#     参数:
#         df: 输入的DataFrame
#         prompt_template: 包含{cut_news}和{sentiment_dict_v2}占位符的模板
#         batch_size: 每次处理的批次大小，默认为1
        
#     返回:
#         处理后的DataFrame，包含预测结果和提取的分数
#     """
#     # 添加结果列
#     if 'lhl_ds_output' not in df.columns:
#         df['lhl_ds_output'] = None
#     if 'findal_score' not in df.columns:
#         df['findal_score'] = None
    
#     # 定义正则表达式模式，用于提取情绪得分
#     score_pattern = r'{"情绪得分"\s*:\s*"([^"]+)"}'
    
#     # 创建进度条
#     total_rows = len(df)
#     print(f"开始处理 {total_rows} 行数据...")
    
#     # 使用tqdm显示进度
#     for i in tqdm(range(total_rows)):
#         row = df.iloc[i]
        
#         try:
#             # 替换模板中的占位符
#             current_prompt = prompt_template.replace("{cut_news}", str(row['cut_news'])).replace("{sentiment_dict_v2}", str(row['sentiment_dict_v2']))
            
#             # 调用预测函数
#             prediction = predict(current_prompt, temperature=0.3)  # 降低温度以增加确定性
            
#             # 保存完整输出
#             df.at[i, 'lhl_ds_output'] = prediction
            
#             # 提取情绪得分
#             match = re.search(score_pattern, prediction)
#             if match:
#                 score_str = match.group(1)
#                 try:
#                     # 尝试转换为浮点数
#                     score = float(score_str)
#                     df.at[i, 'findal_score'] = score
#                 except ValueError:
#                     # 如果无法转换为浮点数，保留原始字符串
#                     df.at[i, 'findal_score'] = score_str
#             else:
#                 df.at[i, 'findal_score'] = None
                
#             # 每批次处理完成后清理CUDA缓存
#             if (i + 1) % batch_size == 0:
#                 torch.cuda.empty_cache()
                
#         except Exception as e:
#             print(f"处理第 {i} 行时出错: {str(e)}")
#             df.at[i, 'lhl_ds_output'] = f"ERROR: {str(e)}"
#             df.at[i, 'findal_score'] = None
            
#             # 错误后清理CUDA缓存
#             torch.cuda.empty_cache()
            
#             # 出错后暂停短暂时间，让GPU冷却
#             time.sleep(1)
    
#     print("处理完成！")
#     return df

# # 使用示例
# if __name__ == "__main__":
#     # 加载模型（仅需一次）
#     load_model()
    
#     # 读取数据
#     df = pd.read_csv("your_data.csv")
    
#     # 设置prompt模板
#     prompt_template = """
#     **角色定义**
#     擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分：
#     以下是新闻文本：
#     <新闻文本>
#     {cut_news}
#     </新闻文本>

#     以下是情绪词典：
#     <情绪词典>
#     {sentiment_dict_v2}
#     </情绪词典>

#     **核心任务**
#     1. 分析新闻整体内容情绪
#     2. 匹配情绪词典关键词
#     3. 五档制评分（-1/-0.5/0/0.5/1）

#     **评分逻辑**
#     1. 语义匹配：忽略偏离词汇，保留有效情绪词
#     2. 评分验证：整体语义优先，情绪词辅助修正

#     **输出规范**
#     - 五档制强制选择
#     - 格式：{"情绪得分":"..."}

#     （情绪词典分值仅作语义方向参考）
#     """
    
#     # 处理数据
#     processed_df = process_dataframe(df, prompt_template, batch_size=5)
    
#     # 保存结果
#     processed_df.to_csv("processed_data.csv", index=False)

### **wxy测试用，记得改prompt**

In [ ]:
print(
    
    """
    <｜begin▁of▁sentence｜>\n**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分：\n以下是新闻文本：\n<新闻文本>\n也有券商迅速发声，大有代替比亚迪“辟谣”之势\n</新闻文本>\n\n以下是情绪词典：\n</情绪词典>\nassertable:0.2, presumptively:0.0\n</情绪词典>\n\n**核心任务**\n1. 分析新闻整体内容情绪\n2. 匹配情绪词典关键词\n3. 五档制评分（-1/-0.5/0/0.5/1）\n\n**评分逻辑**\n1. 语义匹配：忽略偏离词汇，保留有效情绪词\n2. 评分验证：整体语义优先，情绪词辅助修正\n\n**精简示例**\n示例1（积极）：\n新闻：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一\n情绪词：profitability:0.6,profit:0.8\n理由:①语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破；②关键情绪词验证：“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。\n情绪得分:1\n\n示例2（比较积极）：\n新闻：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力\n情绪词：mispricing:-0.4,advantage:0.7\n理由:①语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号；②关键情绪词验证：未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。\n情绪得分:0.5\n\n示例3（中性）：\n新闻：美国信奉自由市场经济理念，主张靠无形的手调整经济活动\n情绪词：free:0.2,immateriality:-0.2\n理由:①语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响；②关键情绪词验证：“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0。\n情绪得分:0\n\n示例4（比较消极）：\n新闻：我们投入的前期费用谁来承担\n情绪词：invest:0.3\n理由:①语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪；②关键情绪词验证：“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位-0.5。\n情绪得分:-0.5\n\n示例5（消极）：\n新闻：饱受美国次贷危机冲击的华尔街再次风云突变\n情绪词：crash:-0.9,meltdown:-0.8\n理由:①语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词验证：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分。\n情绪得分:-1\n\n\n**输出规范**\n- 五档制强制选择\n- 格式：{"情绪得分":"..."}\n\n\n（情绪词典分值仅作语义方向参考）\n<think>好的，我需要分析这条新闻的情绪得分。首先，新闻内容是“也有券商迅速发声，大有代替比亚迪‘辟谣’之势。” 这里的关键点在于券商迅速介入并迅速辟谣。\n\n接下来看情绪词典中的关键词，有assertable（0.2）和presumptively（0.0）。新闻中的“迅速”可能对应assertable，但其分数是0.2，属于中性偏积极。而“辟谣”中的“辟”可能对应presumptively，但分值是0.0，中性。\n\n根据评分逻辑，整体语义优先，情绪词辅助修正。新闻中“迅速辟谣”整体传达出信息量大、迅速的事件，带有积极的意味，但情绪词的辅助需要确认。但“迅速”可能对应assertable的中性部分，而“辟谣”中的“辟”可能对应presumptively的中性，所以情绪词的整体贡献可能中性。\n\n不过，新闻中还有“大有代替”这样的表述，可能暗示对比亚迪的取代行为，但取代通常有负面意义，但这里用了“大有代替”，可能隐含积极的策略，比如质疑或质疑。\n\n综合来看，整体语义偏向积极，但情绪词的辅助可能中性，但“大有代替”可能带有负面情绪，但因为用了“大有”可能暗示积极的策略，所以可能整体情绪中性，但分值低。\n\n不过，情绪词典中assertable是0.2，presumptively是0.0，而新闻中的“迅速辟谣”可能对应assertable的中性部分，而“辟谣”中的“辟”对应presumptively的中性，但整体情绪可能偏向积极，但情绪词的辅助可能中性，所以可能评分在中性偏积极。\n\n不过，可能更倾向于积极，因为“迅速辟谣”可能暗示快速纠正问题，但“大有代替”可能暗示竞争加剧，但“代替”本身可能有负面情绪，但用了“大有”可能暗示积极的策略。\n\n但情绪词典中没有直接匹配的词，所以可能需要结合整体语义和情绪词的匹配。整体情绪偏向积极，但情绪词词典中的积极词分值较低，可能评分中性偏积极。\n\n不过，根据示例，比较积极的新闻评分是0.5，而这里可能更偏向积极，但情绪词词典中的积极词分值较低，所以可能评分是0.5。\n\n不过，仔细分析，新闻中的“迅速辟谣”可能暗示快速纠正问题，但“大有代替”可能暗示竞争加剧，但“代替”本身可能有负面情绪，但用了“大有”可能暗示积极的策略，所以可能评分是积极，但情绪词词典中的积极词分值较低，可能评分是0.5。\n\n但更准确的分析是，新闻整体传达积极的信息，特别是“迅速辟谣”和“大有代替”，但情绪词词典中的积极词分值较低，可能评分是0.5。\n\n不过，可能更接近积极，因为“迅速辟谣”暗示快速纠正问题，而“大有代替”可能暗示竞争加剧，但竞争加剧可能有负面情绪，但“代替”本身可能有负面情绪，但用了“大有”可能暗示积极的策略，所以可能评分是积极。\n\n不过，情绪词典中的“assertable”是0.2，属于中性偏积极，而“presumptively”是0.0，中性，但新闻中的“迅速辟谣”可能对应assertable的中性部分，而“辟谣”中的“辟”对应presumptively的中性，但情绪词的辅助可能需要调整，整体情绪可能偏向积极，但评分可能中性偏积极。\n\n综合考虑，情绪得分可能是0.5。\n</think>\n\n{"情绪得分":"0.5"}<｜end▁of▁sentence｜>
    """
)


    <｜begin▁of▁sentence｜>
**角色定义**
擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分：
以下是新闻文本：
<新闻文本>
也有券商迅速发声，大有代替比亚迪“辟谣”之势
</新闻文本>

以下是情绪词典：
</情绪词典>
assertable:0.2, presumptively:0.0
</情绪词典>

**核心任务**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词
3. 五档制评分（-1/-0.5/0/0.5/1）

**评分逻辑**
1. 语义匹配：忽略偏离词汇，保留有效情绪词
2. 评分验证：整体语义优先，情绪词辅助修正

**精简示例**
示例1（积极）：
新闻：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词：profitability:0.6,profit:0.8
理由:①语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破；②关键情绪词验证：“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。
情绪得分:1

示例2（比较积极）：
新闻：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词：mispricing:-0.4,advantage:0.7
理由:①语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号；②关键情绪词验证：未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。
情绪得分:0.5

示例3（中性）：
新闻：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词：free:0.2,immateriality:-0.2
理由:①语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响；②关键情绪词验证：“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immater

In [ ]:
import pandas as pd
from tqdm import tqdm

# 读取CSV文件（需替换为实际路径）
csv_path = "/root/LLM_news_emo_analyze/wxy_use/LLM_test_500_wxy.csv"  # 替换为真实文件路径
#test_500 = pd.read_csv(csv_path,encoding = 'utf 8 sig',lineterminator= '\n')
test_500 = pd.read_csv(csv_path)
#测试


In [ ]:
test_7 = test_500.head(16)

In [ ]:
prompt = """**角色定义**
擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分。

**核心任务**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词
3. 五档制评分（-1/-0.5/0/0.5/1）

**评分逻辑**
1. 语义匹配：忽略偏离词汇，保留有效情绪词
2. 评分验证：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1（积极）：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
输出:<think>语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破。“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。</think> {"情绪得分":"1"}

- 示例2（比较积极）：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
输出:<think>语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号。此外，新闻文本未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。</think> {"情绪得分":"0.5"}

- 示例3（中性）：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
输出:<think>语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响。此外，新闻文本中“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0。</think> {"情绪得分":"0"}

- 示例4（比较消极）：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
输出:<think>语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪。新闻文本中“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位-0.5。</think> {"情绪得分":"-0.5"}

- 示例5（消极）：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
输出:<think>语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词验证：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，符合-1的情绪评分。</think> {"情绪得分":"-1"}


**输出规范**
- 五档制强制选择
- 输出格式：{"情绪得分":"..."}

现在请你开始进行分析，严格按照输出规范输出结果，情绪词典分值仅作语义方向参考：
新闻文本：{cut_news}
情绪词典：{sentiment_dict_v2}
"""


In [ ]:
test_7['FTdeepseek_output'] = None
for index, row in tqdm(test_7.iterrows(), total=test_7.shape[0], desc="处理进度", unit="行"):
    current_cut_news = row['cut_news']
    current_sentiment_dict_v2 = row['sentiment_dict_v2']
    # 使用 replace 方法替换占位符
    filled_prompt = prompt.replace("{cut_news}", current_cut_news).replace("{sentiment_dict_v2}", current_sentiment_dict_v2)
    emotion_score = predict(filled_prompt, return_full=True)
    test_7.at[index, 'FTdeepseek_output'] = emotion_score

/tmp/ipykernel_28718/2677121740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_7['FTdeepseek_output'] = None
处理进度: 100%|██████████| 16/16 [09:00<00:00, 33.75s/行]


In [ ]:
print(test_7['FTdeepseek_output'][3])

<think>好的，我需要分析这段新闻文本的情绪，并给出五档制评分。首先，看看新闻内容：“理论上，市场的过度竞争会导致所有参与者利润不断趋近于零，而生存的本能欲望，极易导致‘经营失德’。” 这里主要讨论的是过度竞争带来的问题，特别是企业经营失德。

接下来，检查情绪词典里的关键词。情绪词典里有mismanagement:-0.5和uncompetitive:-0.3。新闻中提到的是过度竞争导致利润趋近于零，这可能涉及竞争加剧的问题，但没有直接出现“竞争”或“失德”的关键词。不过，这里可能隐含了竞争带来的负面影响，也就是“经营失德”。

然后，按照评分逻辑，先看整体语义。新闻中虽然没有明确的“竞争”或“失德”词汇，但“过度竞争”和“经营失德”都是负面情绪的迹象。整体语义偏向负面，因为竞争加剧可能导致公司失去竞争力，经营失德可能指公司不能维持正常运营。

再结合情绪词典中的关键词，新闻中没有匹配的词，只是在整体语义上暗示了负面后果。因此，情绪得分应该在中性或负面方向。根据示例，中性是0，负面可能在-0.5或更低。但这里没有明显的负面情绪词，只是隐含竞争带来的负面影响，所以可能属于中性。

综上所述，情绪得分应该是0。
</think> {"情绪得分":"0"}<｜end▁of▁sentence｜>


In [ ]:
test_500['FTdeepseek_output'] = None
for index, row in tqdm(test_500.iterrows(), total=test_500.shape[0], desc="处理进度", unit="行"):
    current_cut_news = row['cut_news']
    current_sentiment_dict_v2 = row['sentiment_dict_v2']
    # 使用 replace 方法替换占位符
    filled_prompt = prompt.replace("{cut_news}", current_cut_news).replace("{sentiment_dict_v2}", current_sentiment_dict_v2)
    emotion_score = predict(filled_prompt, return_full=True)
    test_500.at[index, 'FTdeepseek_output'] = emotion_score


处理进度:   0%|          | 0/500 [01:29<?, ?行/s]


KeyboardInterrupt: 

In [ ]:
#并行计算
import torch
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# 定义处理单个项目的函数
def process_single_item(args):
    index, row, prompt_template = args
    current_cut_news = row['cut_news']
    current_sentiment_dict_v2 = row['sentiment_dict_v2']
    filled_prompt = prompt_template.replace("{cut_news}", current_cut_news).replace("{sentiment_dict_v2}", current_sentiment_dict_v2)
    emotion_score = predict(filled_prompt, return_full=True)
    return index, emotion_score

# 准备数据
items_to_process = [(index, row, prompt) for index, row in test_7.iterrows()]

# 初始化结果列
test_7['FTdeepseek_output'] = None

# 使用线程池并行处理
# 调整max_workers数量，通常设置为2-4较为合适，避免GPU内存溢出
with ThreadPoolExecutor(max_workers=8) as executor:
    # 使用tqdm显示进度
    results = list(tqdm(
        executor.map(process_single_item, items_to_process),
        total=len(items_to_process),
        desc="GPU并行处理",
        unit="样本"
    ))

# 更新结果到DataFrame
for index, result in results:
    test_7.at[index, 'FTdeepseek_output'] = result

/tmp/ipykernel_28718/2952616064.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_7['FTdeepseek_output'] = None
GPU并行处理: 100%|██████████| 16/16 [05:58<00:00, 22.42s/样本]


In [ ]:
print(test_7['FTdeepseek_output'][3])

<think>好的，我现在需要分析这条新闻的情绪得分。首先，新闻内容是关于市场的过度竞争会导致所有参与者利润趋近于零，而生存本能欲望会导致“经营失德”。情绪词典里有“mismanagement:-0.5”和“uncompetitive:-0.3”。

首先看整体语义。新闻提到过度竞争会导致利润趋近于零，这可能暗示市场竞争过猛，企业可能陷入困境，甚至可能失去生存的本能。这可能是一个负面的信号，因为过度竞争通常会导致企业亏损，市场不稳定。同时，提到“经营失德”可能意味着企业失去了经营的基本能力，进一步加剧了问题。

接下来检查情绪词典中的关键词。新闻中并没有直接出现“mismanagement”或“uncompetitive”，只是提到了“过度竞争”和“经营失德”。不过，“过度竞争”可能与“mismanagement”相关，但新闻里没有明确的“mismanagement”出现。而“经营失德”更接近“uncompetitive”，因为过度竞争可能导致企业失去经营能力，这属于“uncompetitive”的范畴，对应情绪词-0.3。

但是，新闻里提到的是过度竞争导致企业利润趋近于零，这可能不仅仅是竞争导致的失 Compet，也可能是企业自身的问题，比如管理不善、资源分配不均等。所以虽然没有直接的“uncompetitive”，但整体语义上，过度竞争可能暗示市场竞争过猛，导致企业陷入困境，这可能更偏向于负面情绪。

另外，新闻中没有明显的积极词汇，比如“成功”、“增长”等，更多是负面的。因此，虽然情绪词典里的“uncompetitive”有影响，但新闻里没有直接匹配的词，可能需要结合整体语义来判断。

综合来看，新闻中提到过度竞争导致企业利润趋近于零，这可能暗示市场竞争过猛，企业可能面临生存压力，导致企业失去经营能力，这可能属于负面情绪。但因为新闻里没有明确的“uncompetitive”出现，可能需要结合“uncompetitive”的程度。如果过度竞争导致企业无法维持正常运营，可能属于较严重的负面，但不确定是否达到“uncompetitive”的程度。

不过，新闻里提到的是“经营失德”，这可能更接近“uncompetitiveness”（经营失德）这个方向，但新闻里并没有直接的匹配词。因此，可能需要结合整体语义和情绪词典的参考。

所以，情绪词典里的“uncompet

In [ ]:
import re
test_500['FTdeepseek_output'] = test_500['FTdeepseek_output'].astype(str)
# 定义正则表达式
pattern = r'{"情绪得分":"([-+]?\d+(?:\.\d+)?)"}'

# 定义提取函数
def extract_score(text):
    match = re.search(pattern, text)
    if match:
        return float(match.group(1))
    return None

# 创建新列并保存提取结果
test_500['FTdeepseek'] = test_500['FTdeepseek_output'].apply(extract_score)

In [ ]:
import re
# 定义正则表达式
pattern = r'{情绪得分:([-+]?\d+(?:\.\d+)?)}'

def extract_score(text):
    if pd.isnull(text) or not isinstance(text, str):
        return None
    match = re.search(pattern, text)
    if match:
        return float(match.group(1))
    return None

# 创建新列并保存提取结果
test_500['deepseek'] = test_500['deepseekk_content'].apply(extract_score)


In [ ]:
test_500.to_csv("/root/LLM_news_emo_analyze/wxy_use/deepseek_test_500_wxy_0420.csv", index=False)

In [ ]:
print(test_7['FTdeepseek_output'])

In [ ]:
# 处理qwen_output列
test_7["FTdeepseek_output"] = None
for index, row in tqdm(test_7.iterrows(), total=test_7.shape[0], desc="处理进度", unit="行"):
    prompt = row["qwen_input"]
    model_response = predict(prompt)
    test_7.at[index, "FTdeepseek_output"] = model_response

/tmp/ipykernel_66903/3480445426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_7["FTdeepseek_output"] = None
处理进度: 100%|██████████| 6/6 [03:10<00:00, 31.70s/行]


In [ ]:
print(test_7["FTdeepseek_output"].head(1))

0    {'thinking_process': None, 'final_answer': '分）...
Name: FTdeepseek_output, dtype: object


In [ ]:
import pandas as pd
test = pd.read_csv("/root/LLM_news_emo_analyze/wxy_use/deepseek_test_500_wxy_0415.csv", encoding='utf_8_sig', lineterminator='\n')

In [ ]:
test.iloc[-1,-3]

'{\'thinking_process\': \'好的，我现在需要分析这条新闻的情绪得分。首先，新闻内容是：“以前，针对原材料价格低、企业生产规模小、铝型材巿场受塑钢冲击大的特点，把经营模式定位为薄利多销型。” \\n\\n首先，我要理解这句话的整体意思。这里在讨论过去的企业经营模式，指出原材料价格低、企业生产规模小、铝型材市场受塑钢冲击大的特点，然后将经营模式定位为薄利多销型。薄利多销通常意味着卖价较低，但利润相对较低，可能带来市场波动。\\n\\n接下来，检查情绪词典中的关键词。情绪词典里有undercuts:-0.3和undercut:-0.3。需要看看新闻中有没有这些词。\\n\\n新闻中提到“薄利多销型”，这里的“薄利”对应undercut，但“多销”是“undercut”的反义词，也就是“多销”对应undercut的正向。不过在词典里，“undercut”是-0.3，而“undercuts”是-0.3。这里句子中并没有直接出现“undercut”或“undercuts”，而是“薄利多销型”，可能只是描述经营模式，没有明确的负面词汇。\\n\\n接下来，分析整体语义。新闻提到的是过去的企业结构问题，强调薄利多销，但没有直接给出负面情绪的词汇，比如“亏本”或“亏损”。而是强调当前的市场状况，比如原材料价格低、企业生产规模小，铝型材市场受塑钢冲击大，这些可能暗示市场波动，但并没有明显的负面情绪如“亏损”或“负面竞争”。\\n\\n所以，整体来看，新闻主要是在陈述过去的问题，没有明显的负面词汇，也没有明显的积极情绪。因此，情绪得分应该在中性范围内，也就是0。\\n\\n再检查一下情绪词典的匹配。虽然没有直接出现“undercuts”或“undercut”，但“薄利多销型”可能暗示一些负面因素，但并没有明确的负面词汇。因此，整体上情绪应该是中性的。\\n\\n综上所述，新闻的情绪得分应该是0。\', \'final_answer\': \'{"情绪得分":"0"}<｜end▁of▁sentence｜>\', \'full_response\': \'<｜begin▁of▁sentence｜>\\n**角色定义**\\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分：\\n以下是新闻文本：\\n<新闻文本>\\n以前，针对原材料价格低、

In [ ]:
test.to_csv('output.csv',encoding = 'utf_8_sig',index = False)

In [ ]:
import pandas as pd

In [ ]:
test = pd.read_csv("/root/LLM_news_emo_analyze/wxy_use/deepseek_test_500_wxy_0420.csv", encoding='utf_8_sig', lineterminator='\n')

In [ ]:
test

,DeclareDate,Title,NewsContent,NewsSource,yearmonth,stratum,topic,similarity,prompt,cut_news,sentiment_dict,sentiment_dict_v2,qwen_input,deepseekk_content,reasoning_content,Unnamed: 15,人工标注,FTdeepseek_output,FTdeepseek,deepseek
0,2018/1/2 0:00,元旦谁抢中国游客多？日本韩国，一个笑了一个哭了,“世界那么大，我想去看看。”这个元旦小长假，不少中国人就选择了境外游迎接跨年。 日前，携...,每日经济新闻,18-Jan,2018-01-02 00:00:00_每日经济新闻,“消费”,0.611313,新闻文本是##“携程跟团游、自由行数据，香港、曼谷、东京、新加坡、大阪、长滩岛、台北、芽庄、...,携程跟团游、自由行数据，香港、曼谷、东京、新加坡、大阪、长滩岛、台北、芽庄、迪拜、伊斯坦布尔...,"海外的/国外的(overseas):0.0,外国的/国外的(foreign):0.0","overseas:0.0, foreign:0.0",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:0},好的，我现在需要分析这条新闻的情绪得分。首先，看看新闻内容。主要讲的是携程的跟团游和自由行数...,NaN,0.0,<think>好的，我现在需要分析给定的新闻文本，并根据提供的情绪词典进行情绪评分。首先，我...,0.0,0.0
1,2012/2/12 0:00,让农民在家门口享受到金融服务,湖北省洪湖市周坊村，60岁的张远航如今在本村就能领取到养老保险金，是农业银行在村里设置的代...,经济日报,12-Feb,2012-02-12 00:00:00_经济日报,“消费”,0.570472,新闻文本是##“而福喜集团总部所在地的美国，更被称作是全球食品安全监管体系最完善的国家之一，...,而福喜集团总部所在地的美国，更被称作是全球食品安全监管体系最完善的国家之一，其着力于事前监管...,"完美的/完善的(perfect):1.0,食物/食品(food):0.0","perfect:1.0, food:0.0",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:0},好的，我现在需要处理这个用户的查询。首先，用户给了一个新闻文本，要求分析市场情绪，并根据给定...,NaN,0.0,<think>好的，我现在需要分析这段新闻的情绪，并给出五档制评分。首先，我得仔细阅读新闻内...,0.5,0.0
2,2020/9/1 0:00,人民币汇率为何持续走强,8月31日，中国人民银行授权中国外汇交易中心公布，人民币对美元汇率中间价报6.8605，上...,经济日报,20-Sep,2020-09-01 00:00:00_经济日报,“消费”,0.485701,新闻文本是##“其中指出，2021年全球受饥饿影响的人数已达8.28亿，较2020年增加约4...,其中指出，2021年全球受饥饿影响的人数已达8.28亿，较2020年增加约4600万，自新冠...,"供应过剩(oversupply):-0.2,过量的/过剩的(excessive):-0.3","oversupply:-0.2, excessive:-0.3",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:-1},好的，我现在要分析这条新闻的情绪得分。首先，新闻内容是关于2021年全球饥饿人数增加的情况，...,NaN,-1.0,<think>好的，我现在需要分析这条新闻的情绪得分。首先，新闻内容是关于全球受饥饿影响的人...,0.0,-1.0
3,2017/6/26 15:39,钮文新：不能宽容高杠杆的源头,文| CCTV证券资讯频道总编辑 钮文新\n责编：赵泽\n（本文刊发于《中国经济周刊》201...,中国经济周刊,17-Jun,2017-06-26 15:39:00_中国经济周刊,“消费”,0.456633,新闻文本是##“理论上，市场的过度竞争会导致所有参与者利润不断趋近于零，而生存的本能欲望，极...,理论上，市场的过度竞争会导致所有参与者利润不断趋近于零，而生存的本能欲望，极易导致“经营失德”,"管理不善/管理失误(mismanagement):-0.5,无竞争力的(uncompetit...","mismanagement:-0.5, uncompetitive:-0.3",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:-1},好的，我需要分析这段新闻文本的情绪，并根据给定的情绪词典进行评分。首先，看看新闻内容：“理论...,NaN,-1.0,<think>好的，我现在需要分析给定的新闻文本的情绪得分。首先，根据用户提供的角色定义，我...,0.0,-1.0
4,2024/5/30 14:23,​打通供应链，打造共赢链,《中国经济周刊》 记者 侯隽 | 北京报道\n供应链话题正在被高度关注。\n受贸易保护主义、...,中国经济周刊,24-May,2024-05-30 14:23:00_中国经济周刊,“生产投资”,0.676999,新闻文本是##“推动多边贸易体制，共同维护产业链供应链畅通\n我国是全世界唯一拥有联合国产业...,推动多边贸易体制，共同维护产业链供应链畅通\n我国是全世界唯一拥有联合国产业分类中全部工业门...,"工业/产业(industry):0.0,工业的/产业的(industrial):0.0","industry:0.0, industrial:0.0",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:0.5},好的，我来仔细分析一下这个新闻文本。首先，新闻内容主要讲的是中国作为唯一拥有全部工业门类的国...,NaN,0.0,<think>好的，我需要分析这段新闻文本的情绪，并根据给定的情绪词典和评分逻辑给出五档制评...,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2024/11/28 15:11,产融合作赋能新型工业化,《中国经济周刊》 记者 孙庭阳 郭霁瑶 |北京报道\n制造业，是立国之本、强国之基。当前我国...,中国经济周刊,24-Nov,2024-11-28 15:11:00_中国经济周刊,“生产投资”,0.538206,新闻文本是##“”上述负责人介绍，“基金额度循环使用，即企业结清贷款后，相应额度会释放出来，...,”上述负责人介绍，“基金额度循环使用，即企业结清贷款后，相应额度会释放出来，供有需要的企业继...,"基金/资金(fund):0.1,基本的/根本的(fundamental):0.3","fund:0.1, fundamental:0.3",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:0},好的，我现在需要处理这个市场情绪分析的任务。首先，我得仔细阅读用户提供的所有信息，确保自己理...,NaN,0.0,NaN,NaN,0.0
496,2007/1/14 0:00,村里超市开业了,1月7日，星期天，记者在山东省文登市采访。雪后路滑，气温较低，但却是晴朗朗的好天气。听说文登...,经济日报,7-Jan,2007-01-14 00:00:00_经济日报,“消费”,0.547639,新闻文本是##“现在农村有了超市，好多假货彻底失去了市场”##，包含的情绪词典是##：无销路...,现在农村有了超市，好多假货彻底失去了市场,"无销路的/滞销的(unmarketable):-0.5,滞销(unsellable):-0.4","unmarketable:-0.5, unsellable:-0.4",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:1},好的，我需要分析这条新闻的情绪。新闻内容是：“现在农村有了超市，好多假货彻底失去了市场。”首...,NaN,1.0,NaN,NaN,1.0
497,2024/3/30 0:08,如何错误地防范金融危机,韩明睿不少大众社科书籍都有一个抓人眼球但不太容易看出是什么主题的书名，副标题则是解释性质的“...,经济观察报,24-Mar,2024-03-30 00:08:00_经济观察报,“生产投资”,0.526640,新闻文本是##“在全球气候变化挑战日益严峻的大背景下，欧洲迎来绿色转型大潮，其中以法国为代表...,在全球气候变化挑战日益严峻的大背景下，欧洲迎来绿色转型大潮，其中以法国为代表的部分国家将发展...,"气候/环境(climate):0.0,法国的/法语的(french):0.0","climate:0.0, french:0.0",**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,{情绪得分:0},好的，我需要分析给定的新闻文本的情绪得分。首先，